# BERT 전이학습

### 데이터 준비

In [1]:
from tensorflow.keras.utils import get_file

ratings_train_path = get_file("rating_train.txt", 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
ratings_test_path = get_file("rating_test.txt", 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

14628807/14628807 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4893335/4893335 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep='\t')
ratings_test_df = pd.read_csv(ratings_test_path, sep='\t')

display(ratings_train_df.head())
display(ratings_test_df.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [3]:
ratings_train_df = ratings_train_df.dropna(how='any')
ratings_test_df = ratings_test_df.dropna(how='any')

display(ratings_train_df.info())
display(ratings_test_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


None

In [4]:
ratings_train_df = ratings_train_df.sample(n=15000, random_state=0) # 학습데이터 15000개 무작위 추출(재현성 고정)
ratings_test_df = ratings_test_df.sample(n=5000, random_state=0)

# 라벨 (0/1) 개수 분포 확인
ratings_train_df['label'].value_counts(), ratings_test_df['label'].value_counts()

(label
 0    7512
 1    7488
 Name: count, dtype: int64,
 label
 0    2532
 1    2468
 Name: count, dtype: int64)

In [5]:
# 텍스트/라벨을 리스트로 변환 (학습/테스트 분리)
X_train =  ratings_train_df['document'].values.tolist() # 학습 입력 문장(document) 컬럼을 리스트로 변환
y_train = ratings_train_df['label'].values.tolist()

X_test = ratings_test_df['document'].values.tolist()    # 테스트 입력 문장(document) 컬럼을 리스트로 변환
y_test = ratings_test_df['label'].values.tolist()

len(X_train), len(y_train), len(X_test), len(y_test),

(15000, 15000, 5000, 5000)

### 토크나이저/모델 준비

- bert 한국어 버전 사전학습 모델 klue/bert-base

In [6]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('klue/bert-base')             # 사전학습 KLUE BERT 모델 로드 (가중치 포함)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')     # 사전학습 KLUE BERT 토크나이저 로드 (토큰화 규칙/어휘)

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
tokenizer.model_max_length  # 512 토큰을 넘으면 토큰화시 자름

512

In [8]:
X_train = tokenizer(X_train, padding=True, truncation=True) # 가장 긴 문장을 가진 토큰 기준으로 맞춰진다.
X_test = tokenizer(X_test, padding=True, truncation=True)

X_train[:5], X_test[:5]

([Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])],
 [Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_

In [9]:
print(X_train['input_ids'][0])          # 토큰 -> 정수ID로 변환된 시퀀스
print(X_train['attention_mask'][0])     # 실제 토큰=1, 패딩=0으로 구분한 마스크
print(X_train['token_type_ids'][0])     # 문장 구분 ID(단일 문장은 모두 0)

[2, 1800, 2178, 860, 3629, 16516, 2031, 18, 18, 18, 14242, 2205, 2062, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 데이터 파이프라인 생성

In [10]:
type(X_train)   # tokenizer 반환 객체 타입 확인

transformers.tokenization_utils_base.BatchEncoding

In [11]:
# Tensorflow의 tf.datas.Dataset 생성(BERT 입력 + 라벨 묶ㄱ)
import tensorflow as tf

trian_ds = tf.data.Dataset.from_tensor_slices((dict(X_train),y_train))  # 토큰화 dict + 라벨 = Dataset변환(학습용)
test_ds = tf.data.Dataset.from_tensor_slices((dict(X_test),y_test))     # 토큰화 dict + 라벨 = Dataset변환(테스트용)

In [12]:
# 학습 : 셔플 -> 배치 -> 프리배치로 파이프라인 최적화
train_dataset = trian_ds.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
# 테스트 : 셔플 -> 배치 -> 프리배치로 파이프라인 최적화
test_dataset = test_ds.batch(64).prefetch(tf.data.AUTOTUNE)

.prefetch(tf.data.AUTOTUNE) : 계산이랑 데이터 준비를 겹쳐서 미리 준비해둔다. (적당한 갯수는 Tensorflow에서 알아서 계산한다.)
-> [데이터 로딩 + 모델 계산] -> [데이터 로딩 + 모델 계산]

In [13]:
# KLUE BERT 분류 모델 로드
from transformers import TFBertForSequenceClassification

# 2클래스 분류용 TF BERT 로드(Pytorch 가중치 변환)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base" ,num_labels=2, from_pt = True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should p

In [14]:
# BERT 파인튜닝 설정 및 학습 (옵티마이저/컴파일/학습)
from transformers import create_optimizer

num_train_steps = len(train_dataset) * 5        # 전체학습의 step 수(steps_per_epoch * 5(epochs))
num_warnup_steps = int(num_train_steps * 0.1)   # warmup step 수(전체의 10%)

# Transformers 권장 스케쥴 포함 옵티마이저 생성
optimizer, _=create_optimizer(
    init_lr = 5e-5,
    num_train_steps=num_train_steps,            # 총 학습 step 수
    num_warmup_steps=num_warnup_steps,          # warmup step 수
    weight_decay_rate=0.1                       # weight decay 비율(가중치 감쇠 : 과적합 완화)
)

# 모델 학습설정
model.compile(
    optimizer=optimizer,
    loss=model.hf_compute_loss, # HF 제공 loss 계산 함수
    metrics=['accuracy']
)

# 모델 학습
model.fit(train_dataset,epochs=5,
            validation_data=test_dataset,
            batch_size=64
)

Epoch 1/5
235/235 [==============================] - 504s 2s/step - loss: 0.3988 - accuracy: 0.8140 - val_loss: 0.3084 - val_accuracy: 0.8656
Epoch 2/5
235/235 [==============================] - 448s 2s/step - loss: 0.2208 - accuracy: 0.9134 - val_loss: 0.3113 - val_accuracy: 0.8694
Epoch 3/5
235/235 [==============================] - 447s 2s/step - loss: 0.1100 - accuracy: 0.9619 - val_loss: 0.4047 - val_accuracy: 0.8710
Epoch 4/5
235/235 [==============================] - 447s 2s/step - loss: 0.0470 - accuracy: 0.9852 - val_loss: 0.5229 - val_accuracy: 0.8736
Epoch 5/5
235/235 [==============================] - 447s 2s/step - loss: 0.0272 - accuracy: 0.9920 - val_loss: 0.5609 - val_accuracy: 0.8746


- 워밍업(warmup) 스탭이란 학습 초반에는 학습률을 바로 크게 쓰지않고. 작은 값에서 ㅅ작해서 일정 스탭동안 점진적으로 올리는 구간
    - 스탭 * 0.1 => 전체 스탭에서 10%동안 LR를 천천히 올리는 설정
    - 동작 과정 : num_warmup_steps 동안 LR를 0 -> init_lr로 선형으로 증가시키고, 워밍업이 끝나면 LR가 점점 감소한다.
    - 사용 이유 : BERT 같은 사전학습 모델이 파인튜닝시에 초반에 LR가 크면 가중치가 갑자기 크게 바뀌어서 학습이 불안정/발산이 일어나 성능이 흔들릴 가능성이 있다.

In [15]:
# 파인튜닝 모델/토크나이저 저장
model.save_pretrained('nsmc_model/bert-base')       # 학습된 모델 가중치/설정(config) 저장
tokenizer.save_pretrained('nsmc_model/bert_base')   # 토크나이저 파일(vocab, tokenizer_config 등) 저장

('nsmc_model/bert_base/tokenizer_config.json',
 'nsmc_model/bert_base/special_tokens_map.json',
 'nsmc_model/bert_base/vocab.txt',
 'nsmc_model/bert_base/added_tokens.json',
 'nsmc_model/bert_base/tokenizer.json')

### 추론

In [16]:
# 텍스트 분류 라벨 값 지정
model.config.id2label = {
    0 : '부정',
    1 : '긍정'
}

In [22]:
# 감성분석 파이프라인 생성
from transformers import TextClassificationPipeline

# 입력 텍스트 -> 토큰화 -> 모델 추론 -> 라벨/점수 반환 파이프라인
sentment_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,        # 사용할 토크나이저
    model=model,                # 사용할 분류 모델
    framework="tf",                # TensorFlow 기반 모델 사용
    return_all_scores=True      # 모든 라벨 점수(확률/스코어) 반환
)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Device set to use 0


In [23]:
sentment_classifier("인생 영화를 찾았습니다!!!")   # 입력 문장에 대해 부정/긍정 점수 예측

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


[[{'label': '부정', 'score': 0.003968074452131987},
  {'label': '긍정', 'score': 0.9960319399833679}]]

# HuggingFace

### push

In [25]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

In [26]:
# Hugging face Hub에 모델/토크나이저 업로드
REPO_NAME = 'bert-base-nsmc' # Hun에 생성/업로드 진행할 모델 레포지토리

model.push_to_hub(REPO_NAME, use_auth_token=HF_TOKEN, use_temp_dir=True)      # 학습된 모델을 hub repo로 업로드(토큰으로 인증, 임시폴더 사용)
tokenizer.push_to_hub(REPO_NAME, use_auth_token=HF_TOKEN, use_temp_dir=True)  # 토크나이저 파일을 hub repo fh djqfhem(토큰으로 임증, 임시폴더 사용)

/usr/local/lib/python3.12/dist-packages/transformers/modeling_tf_utils.py:3168: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  /tmp/tmpvqad64an/tf_model.h5:   0%|          |  557kB /  443MB            

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hyJung2247/bert-base-nsmc/commit/ad0c320e3ce71c8a7805e0641b41021d7bdb2bec', commit_message='Upload tokenizer', commit_description='', oid='ad0c320e3ce71c8a7805e0641b41021d7bdb2bec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hyJung2247/bert-base-nsmc', endpoint='https://huggingface.co', repo_type='model', repo_id='hyJung2247/bert-base-nsmc'), pr_revision=None, pr_num=None)

In [28]:
# Hugging face Hub에 모델/토크나이저 로드
from transformers import AutoTokenizer, AutoModel

REPO_NAME = 'hyJung2247/bert-base-nsmc'

tokenizer = AutoTokenizer.from_pretrained(REPO_NAME)        # Hub repo에서 토크나이저 다운로드 후 로드
mpdel = AutoModel.from_pretrained(REPO_NAME, from_tf=True)  # Hub repo에서 TR로 저장된 가중치를 읽어 AutoModel로 변환 로드

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/443M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2446: UserWarning: for embeddings.word_embeddings.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2446: UserWarning: for embeddings.position_embeddings.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2446: UserWarning: for embeddings.token_type_embeddings.weight: copying from a non-meta parameter in the checkpoint t

In [29]:
# pipeline(model='repo_id')이 내부에서 Pytorch/TF 둘다 시도 => TF만 있는 환경에서는 에러가 발생할 가능성이 높음
from transformers import pipeline

sentment_classifier = pipeline('text-classification', model=REPO_NAME)

ValueError: Could not load model hyJung2247/bert-base-nsmc with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSequenceClassification'>, <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>, <class 'transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/transformers/pipelines/base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^

In [34]:
# pipeine 생성 + TF 분류 모델 로드 클래스
from transformers import pipeline, TFAutoModelForSequenceClassification

# Hub의 TF 가중치로 분류 모델 로드
model = TFAutoModelForSequenceClassification.from_pretrained(REPO_NAME)

# TF 기반 텍스트 분류 파이프라인 생성
sentment_classifier = pipeline(
    'text-classification',    # 작업 유형 : 텍스트 분류
    model=model,              # 로다한 TF 분류 모델
    tokenizer=tokenizer,       # 로드한 토크나이저
    framework = 'tf'          # 프레임워크를 TF로 고정(추론/로드시 충돌 방지)
)


Some layers from the model checkpoint at hyJung2247/bert-base-nsmc were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at hyJung2247/bert-base-nsmc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Device set to use 0


In [43]:
# 감성분석 추론 실행
sentment_classifier([
    '한국 영화는 이제 한물 갔어!',
    '어쩌라고 나는 재미있던데? ㅋㅋㅋ',
    '혜수 언니 사랑해요',
    '평생 잊지 못할 영화를 만났어요... 감동',
    '으앙 쥬금'
])

[{'label': '부정', 'score': 0.9996190071105957},
 {'label': '부정', 'score': 0.8733336329460144},
 {'label': '긍정', 'score': 0.9525501132011414},
 {'label': '긍정', 'score': 0.9990559220314026},
 {'label': '부정', 'score': 0.744641900062561}]